# Pre-processing - Get synchronization times

This notebook create a file with times of the stimuli for a particular record.

In [24]:
import sys
sys.path.append('../src')
from lib.utils import checkDirectory

from os import path, mkdir
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


experiment_name = '2018-01-25'
# experiment_name = '2018-04-18'
# experiment_name = '2016-08-11'
# experiment_name = '2016-04-11'
# experiment_name = '2016-09-14'
# experiment_name = 'nn'


# Get synchronization signal


In [6]:
mcd_file = '../data/raw_data/'+experiment_name+'/'+experiment_name+'_analog.mcd'
output_folder = '../data/sync/'+experiment_name+'/'
mcd_channel = 1
sampling_rate = 20000
real_fps = 59.7694
checkDirectory(output_folder)

command_matlab = "\"cd ../src/syncAnalyzer/; syncAnalyzer('../{}','{}','../{}',{},{},{}); quit\""\
                .format( mcd_file, experiment_name, output_folder, mcd_channel,
                        sampling_rate,real_fps)
print(command_matlab)
!matlab -nodesktop -nodisplay -nosplash -r $command_matlab

"cd ../src/syncAnalyzer/; syncAnalyzer('../../data/raw_data/2018-01-25/2018-01-25_analog.mcd','2018-01-25','../../data/sync/2018-01-25/',1,20000,59.7694); quit"


## Read start-end file and create event list
From sync file created above we compute the difference between all frames and we only take the frames with a difference more than one frame to create a event.

In [7]:
# load data
source_folder = '../data/sync/'+experiment_name+'/'
start_end_file = source_folder+'start_end_frames_'+experiment_name+'.txt' 
total_duration_file = source_folder+'total_duration_'+experiment_name+'.txt' 

start_end = np.loadtxt( start_end_file , dtype='int32')-1
total_duration = np.loadtxt( total_duration_file , dtype='int32')-1

start_frame, end_frame = start_end[:,0], start_end[:,1]
diff_time = np.diff( np.concatenate(
            ( np.array([0]), start_frame, np.array([total_duration]) ) ))

dis_frame = np.ceil(sampling_rate/real_fps) #max dist between frames
filter_event = diff_time > dis_frame
end_event_pos = np.where( filter_event == True )[0]

# Select start and end time for each event
start_event = start_frame[ end_event_pos[:-1] ]
end_event = end_frame[ end_event_pos[1:] - 1 ]
n_frames = end_event_pos[1:] - end_event_pos[:-1]

# Add bound condition 
start_event_full = np.concatenate(( np.array([0]), start_event, end_event[-1:] ))
end_event_full = np.concatenate(( start_event[:1], end_event, np.array([total_duration]) ))
n_frames_full = np.concatenate(( np.array([0]), n_frames, np.array([0]) ))

# Create a DataFrame to create a list of event
event_schedule = pd.DataFrame({'start_event': start_event_full,'end_event':end_event_full,
                               'n_frames':n_frames_full})

event_schedule['event_duration'] = (event_schedule['end_event']-
                                    event_schedule['start_event'])/sampling_rate

inter_event_duration = event_schedule[['start_event','end_event']].values / sampling_rate
event_schedule['inter_event_duration'] = pd.Series(inter_event_duration[1:,0] - inter_event_duration[:-1,1])
event_schedule.loc[len(event_schedule)-1,'inter_event_duration'] = 0

### Add repetition frame time  in csv file

In [ ]:
# Load data
repeated_file = source_folder+'repeated_frames_'+experiment_name+'.txt' 
repeated_frame_time = np.loadtxt(repeated_file, dtype='int32')

event_schedule['repeated_frames'] = ''
event_schedule['#repeated_frames'] = 0
events = event_schedule[['start_event','end_event']].values
for kidx, (kstart, kend) in enumerate(events):
    filter_rep = (repeated_frame_time >= kstart) * (repeated_frame_time <= kend)
    event_schedule.loc[kidx,'repeated_frames'] = str(repeated_frame_time[filter_rep])
    event_schedule.loc[kidx,['#repeated_frames']] = len(repeated_frame_time[filter_rep])

# sorted columns
event_schedule = event_schedule[['n_frames','start_event','end_event',
                                 'event_duration','inter_event_duration',

                                 '#repeated_frames','repeated_frames']]

In [23]:
filter_frame = event_schedule['n_frames'] > 10
event_schedule[filter_frame]

,n_frames,start_event,end_event,event_duration,inter_event_duration,#repeated_frames,repeated_frames
6,18612,6038432,12268074,311.48210,8.45740,9306,[ 6038433 6039103 6039772 ... 12266068 12266...
7,2100,12437222,13140115,35.14465,3.99985,0,[]
8,2100,13220112,13923005,35.14465,3.99980,0,[]
9,2100,14003001,14705894,35.14465,3.99985,0,[]
10,2100,14785891,15488783,35.14460,3.99985,0,[]
11,2100,15568780,16271673,35.14465,3.99980,0,[]
12,2100,16351669,17054562,35.14465,3.99985,0,[]
13,2100,17134559,17837452,35.14465,3.99980,0,[]
14,2100,17917448,18620341,35.14465,3.99985,0,[]
15,2100,18700338,19403231,35.14465,3.99980,0,[]


## Save csv file

In [30]:
output_folder = '../data/sync/'+experiment_name+'/schedule_event/'
checkDirectory(output_folder)
event_schedule.to_csv(output_folder+'schedule_event_'+experiment_name+'_.csv',index=False)

**Note:** The times in event_duration have small differences with the theoric times, it's because the theorical FPS of the projector is different to the real FPS.   

For example, projector show theoricly 60 FPS but the real FPS is 59.7523. Please ckeck it in the log file.
* 2100 images to 60 [fps] = 35 [s]
* 2100 images to 59.7523 [fps] = 35.145 [s]
The differences could be significant!

## Create separated sync files for each event


In [ ]:
source_folder = '../data/sync/'+experiment_name+'/'
output_folder = '../data/sync/'+experiment_name+'/schedule_event/times/'
checkDirectory(output_folder)

event_file = source_folder+'schedule_event/schedule_event_'+experiment_name+'.csv'
start_end_file = source_folder+'start_end_frames_'+experiment_name+'.txt' 

# Load data
event_schedule_def = pd.read_csv(event_file)
start_end = np.loadtxt( start_end_file , dtype='int32')-1
start_frame, end_frame = start_end[:,0], start_end[:,1]

pointer = start_end
for kidx, (kstart,kend) in enumerate(event_schedule_def[['start_event','end_event']].values):
    filter_frame = (pointer[:,0] >= kstart) * (pointer[:,0] < kend) 
    start_end_event = pointer[filter_frame]
    start_end_event[-1,1] = kend
    np.savetxt(output_folder+'{:03d}.txt'.format(kidx),start_end_event,fmt='%d')
    print(output_folder+'{:03d}.txt'.format(kidx))
    pointer = pointer[~filter_frame]
    